In [1]:
import tkinter as tk
from enum import Enum
import re
import pandas
import pandastable as pt
from nltk.tree import *

In [ ]:
class Token_type(Enum):  # listing all tokens type
    program = 1
    implicit = 2
    none = 3
    integer = 4
    real = 5
    Complex = 6
    logical = 7
    Dot = 8
    Semicolon = 9
    EqualOp = 10
    LessThanOp = 11
    GreaterThanOp = 12
    NotEqualOp = 13
    PlusOp = 14
    MinusOp = 15
    MultiplyOp = 16
    DivideOp = 17
    VarDeclOp = 18
    character = 19
    ExclMark = 20
    parameter = 21
    end = 22
    If = 23
    then = 24
    Else = 25
    do = 26
    string = 27
    read = 28
    Print = 29
    LessThanEqualOp = 30
    GreaterThanEqualOp = 31
    EqualEqualOp = 32
    constant = 33
    identifier = 34
    Error = 35
    Comma = 36
    Len = 37
    openParenthesis=38
    closeParenthesis=39


In [ ]:
# Reserved word Dictionary
ReservedWords = {
    "program": Token_type.program,
    "implicit": Token_type.implicit,
    "none": Token_type.none,
    "end": Token_type.end,
    "integer": Token_type.integer,
    "real": Token_type.real,
    "complex": Token_type.Complex,
    "logical": Token_type.logical,
    "character": Token_type.character,
    "parameter": Token_type.parameter,
    "if": Token_type.If,
    "then": Token_type.then,
    "else": Token_type.Else,
    "do": Token_type.do,
    "read": Token_type.read,
    "print": Token_type.Print,
    "len": Token_type.Len
}


In [ ]:
Operators = {
    ".": Token_type.Dot,
    # ";": Token_type.Semicolon,
    "=": Token_type.EqualOp,
    "+": Token_type.PlusOp,
    "-": Token_type.MinusOp,
    "*": Token_type.MultiplyOp,
    "/": Token_type.DivideOp,
    "::": Token_type.VarDeclOp,
    "!": Token_type.ExclMark,
    ">": Token_type.GreaterThanOp,
    "<": Token_type.LessThanOp,
    "<=": Token_type.LessThanEqualOp,
    ">=": Token_type.GreaterThanEqualOp,
    "/=": Token_type.NotEqualOp,
    "==": Token_type.EqualEqualOp,
    ",": Token_type.Comma,
    "(":Token_type.openParenthesis,
    ")":Token_type.closeParenthesis
}

## Operator Precedence
# '*'    '/'    '+'    '-'    '>'    '<'    '<='    '>='    '=='    '/='


In [ ]:
# class token to hold string and token type
class token:
    def __init__(self, lex, token_type):
        self.lex = lex
        self.token_type = token_type

    def to_dict(self):
        return {
            'Lex': self.lex,
            'token_type': self.token_type
        }


In [ ]:
Tokens=[]
Errors=[]

In [ ]:
def find_token(text):
    lexems = text.split()
    for le in lexems:
        if (le in ReservedWords):
            new_token = token(le, ReservedWords[le])
            Tokens.append(new_token)
        elif (le in Operators):
            new_token = token(le, Operators[le])
            Tokens.append(new_token)
        elif (re.match("^\d+(\.[0-9]*)?$", le)):
            new_token = token(le, Token_type.constant)
            Tokens.append(new_token)
        elif (re.match("^([a-zA-Z][a-zA-Z0-9]*)$", le)):
            new_token = token(le, Token_type.identifier)
            Tokens.append(new_token)
        elif (re.match("^\"[\w. ]+\"$", le) or re.match("^\'[\w. ]+\'$", le)):
            new_token = token(le, Token_type.string)
            Tokens.append(new_token)
        else:
            new_token = token(le, Token_type.Error)
            Errors.append("Lexical error  " + le)


In [ ]:
def Match(TT,i) :
    out = dict()
    if(i<len(Tokens)) :
        TokDict=Tokens[i].to_dict()
        if(TokDict['token_type'] == TT) :
            i+=1
            out['node'] = [TokDict['Lex']]
            out['index']=i
            return out
        else:
            out['node']=['error']
            out['index']=i+1
            Errors.append("Syntax Error: "+TT['Lex'])
            return out
    else :
        out['node']=['error']
        out['index']=i+1
        return out


In [ ]:
def Parse():
    i = 0
    ParseChildren = []
    ProgramStartDict = ProgramStart(i)


def ProgramStart(i):
    ProgramStart_Dict = dict()
    ProgramStart_Children = []
    Dict1 = ProgramUnit(i)
    ProgramStart_Children.append(Dict1['node'])
    Dict2 = ProgramStart2(Dict1['index'])
    ProgramStart_Children.append(Dict2['node'])
    ProgramStart_Node = Tree("ProgramStart", ProgramStart_Children)
    ProgramStart_Dict['node'] = ProgramStart_Node
    ProgramStart_Dict['index'] = Dict2['index']
    return ProgramStart_Dict


def ProgramStart2(i):
    ProgramStart2_Dict = dict()
    ProgramStart2_Children = []
    if i < len(Tokens):
        Temp = Tokens[i].to_dict()
        if Temp['token_type'] == Token_type.program:
            Dict1 = ProgramUnit(i)
            ProgramStart2_Children.append(Dict1['node'])
            Dict2 = ProgramStart2(Dict1['index'])
            ProgramStart2_Children.append(Dict2['node'])
            ProgramStart2_Node = Tree("ProgramStart2", ProgramStart2_Children)
            ProgramStart2_Dict['node'] = ProgramStart2_Node
            ProgramStart2_Dict['index'] = Dict2['index']
            return ProgramStart2_Dict
        else:
            match1 = Match(Token_type.Error, i)
            ProgramStart2_Children.append(match1['node'])
            ProgramStart2_Node = Tree("ProgramStart2", ProgramStart2_Children)
            ProgramStart2_Dict['node'] = ProgramStart2_Node
            ProgramStart2_Dict['index'] = match1['index']
            return ProgramStart2_Dict

    else:

        ProgramStart2_Dict['node'] = None
        ProgramStart2_Dict['index'] = i
        return ProgramStart2_Dict


def ProgramUnit(i):
    ProgramUnit_dict = dict()
    ProgramUnit_children = []
    dict1 = Header(i)
    ProgramUnit_children.append(dict1['node'])
    dict2 = Block(dict1['index'])
    ProgramUnit_children.append(dict2['node'])
    dict3 = Footer(dict2['index'])
    ProgramUnit_children.append(dict3['node'])
    ProgramUnit_node = Tree("ProgramUnit", ProgramUnit_children)
    ProgramUnit_dict['node'] = ProgramUnit_node
    ProgramUnit_dict['index'] = dict3['index']
    return ProgramUnit_dict


def Header(i):
    Header_dict = dict()
    Header_children = []
    match1 = Match(Token_type.program, i)
    Header_children.append(match1['node'])
    match2 = Match(Token_type.identifier, match1['index'])
    Header_children.append(match2['node'])
    Header_node = Tree("Header", Header_children)
    Header_dict['node'] = Header_node
    Header_dict['index'] = match2['index']
    return Header_dict


def Block(i):
    Block_dict = dict()
    Block_children = []
    match1 = Match(Token_type.implicit, i)
    Block_children.append(match1['node'])
    match2 = Match(Token_type.none, match1['index'])
    Block_children.append(match2['node'])
    dict3 = TypeDecls(match2['index'])
    Block_children.append(dict3['node'])
    dict4 = Statements(dict3['index'])
    Block_children.append(dict4['node'])
    Block_node = Tree("Block", Block_children)
    Block_dict['node'] = Block_node
    Block_dict['index'] = dict4['index']
    return Block_dict


def Footer(i):
    Footer_dict = dict()
    Footer_children = []
    match1 = Match(Token_type.end, i)
    Footer_children.append(match1['node'])
    match2 = Match(Token_type.program, match1['index'])
    Footer_children.append(match2['node'])
    match3 = Match(Token_type.identifier, match2['index'])
    Footer_children.append(match3['node'])
    Footer_node = Tree("Footer", Footer_children)
    Footer_dict['node'] = Footer_node
    Footer_dict['index'] = match3['index']
    return Footer_dict


def TypeDecls(i):
    TypeDecls_dict = dict()
    TypeDecls_children = []
    last_index = i
    if i < len(Tokens):
        Temp = Tokens[i].to_dict()
        ############################################
        if Temp['token_type'] in [Token_type.integer, Token_type.real, Token_type.Complex, Token_type.logical, Token_type.character]:
            dict1 = TypeDecl(i)
            TypeDecls_children.append(dict1['node'])
            dict2 = TypeDecls2(dict1['index'])
            TypeDecls_children.append(dict2['node'])
            last_index = dict2['index']
        else:
            match1 = Match(Token_type.Error, i)
            TypeDecls_children.append(match1['node'])
            last_index = match1['index']
    else:
        match1 = Match(Token_type.Error, i)
        TypeDecls_children.append(match1['node'])
        last_index = match1['index']

    TypeDecls_node = Tree("TypeDecls", TypeDecls_children)
    TypeDecls_dict['node'] = TypeDecls_node
    TypeDecls_dict['index'] = last_index
    return TypeDecls_dict


def TypeDecls2(i):
    TypeDecls2_dict = dict()
    TypeDecls2_children=[]
    last_index = i
    if i < len(Tokens):
        Temp = Tokens[i].to_dict()
        ############################################
        if Temp['token_type'] in [Token_type.integer, Token_type.real, Token_type.Complex, Token_type.logical, Token_type.character]:
            dict1 = TypeDecl(i)
            TypeDecls2_children.append(dict1['node'])
            dict2 = TypeDecls2(dict1['index'])
            TypeDecls2_children.append(dict2['node'])
            last_index = dict2['index']
        else:
            match1 = Match(Token_type.Error, i)
            TypeDecls2_children.append(match1['node'])
            last_index = match1['index']
    else:
        match1 = Match(Token_type.Error, i)
        TypeDecls2_children.append(match1['node'])
        last_index = match1['index']

    TypeDecls_node = Tree("TypeDecls", TypeDecls2_children)
    TypeDecls2_dict['node'] = TypeDecls_node
    TypeDecls2_dict['index'] = last_index
    return TypeDecls2_dict

def TypeDecl(i):
    TypeDecl_dict=dict()
    TypeDecl_children=[]
    dict1=DataType(i)
    TypeDecl_children.append(dict1['node'])
    dict2=TypeDecl2(dict1['index'])
    TypeDecl_children.append(dict2['node'])
    TypeDecl_node=Tree("TypeDecl",TypeDecl_children)
    TypeDecl_dict['node']=TypeDecl_node
    TypeDecl_dict['index']=dict2['index']
    return TypeDecl_dict

def TypeDecl2(i):
    TypeDecl2_dict=dict()
    TypeDecl2_children=[]
    last_index=i
    if i < len(Tokens) :
        temp=Tokens[i].to_dict()
        if temp['token_type'] == Token_type.VarDeclOp :
            match1=Match(Token_type.VarDeclOp,i)
            TypeDecl2_children.append(match1['node'])
            dict2=IdentifierList(match1['index'])
            TypeDecl2_children.append(dict2['node'])
            last_index=dict2['index']
        elif temp['token_type'] == Token_type.Comma :
            match1=Match(Token_type.Comma,i)
            TypeDecl2_children.append(match1['node'])
            match2=Match(Token_type.parameter,match1['index'])
            TypeDecl2_children.append(match2['node'])
            match3=Match(Token_type.VarDeclOp,match2['index'])
            TypeDecl2_children.append(match3['node'])
            dict4=NamedConstant(match3['index'])
            TypeDecl2_children.append(dict4['node'])
            last_index=dict4['index']
        else :
            match1=Match(Token_type.Error,i)
            TypeDecl2_children.append(match1['node'])
            last_index=match1['index']
    TypeDecl2_node=Tree("TypeDecl2",TypeDecl2_children)
    TypeDecl2_dict['node']=TypeDecl2_node
    TypeDecl2_dict['index']=last_index
    return TypeDecl2_dict

def DataType(i):
    DataType_dict=dict()
    DataType_children=[]
    last_index=i
    if i < len(Tokens) :
        temp=Tokens[i].to_dict()
        if temp['token_type']==Token_type.integer:
            match1=Match(Token_type.integer,i)
            DataType_children.append(match1['node'])
            last_index=match1['index']
        elif temp['token_type']==Token_type.real:
            match1=Match(Token_type.real,i)
            DataType_children.append(match1['node'])
            last_index=match1['index']
        elif temp['token_type']==Token_type.Complex:
            match1=Match(Token_type.Complex,i)
            DataType_children.append(match1['node'])
            last_index=match1['index']
        elif temp['token_type']==Token_type.logical:
            match1=Match(Token_type.logical,i)
            DataType_children.append(match1['node'])
            last_index=match1['index']
        elif temp['token_type']==Token_type.character:
            dict1=CharacterDType(i)
            DataType_children.append(dict1['node'])
            last_index=dict1['index']
        else :
            match1=Match(Token_type.Error,i)
            DataType_children.append(match1['node'])
            last_index=match1['index']

    else :
        match1=Match(Token_type.Error,i)
        DataType_children.append(match1['node'])
        last_index=match1['index']
    DataType_node=Tree("DataType",DataType_children)
    DataType_dict['node']=DataType_node
    DataType_dict['index']=last_index
    return DataType_dict

def NamedConstant(i):
    NamedConstant_dict=dict()
    NamedConstant_children=[]
    match1=Match(Token_type.identifier,i)
    NamedConstant_children.append(match1['node'])
    match2=Match(Token_type.EqualOp,match1['index'])
    NamedConstant_children.append(match2['node'])
    match3=Match(Token_type.constant,match2['index'])
    NamedConstant_children.append(match3['node'])
    NamedConstant_node=Tree("NamedConstant",NamedConstant_children)
    NamedConstant_dict['node']=NamedConstant_node
    NamedConstant_dict['index']=match3['index']
    return NamedConstant_dict

def CharacterDType(i):
    CharacterDType_dict=dict()
    CharacterDType_children=[]
    match1=Match(Token_type.character,i)
    CharacterDType_children.append(match1['node'])
    dict2=CharacterDType2(match1['index'])
    CharacterDType_children.append(dict2['node'])
    CharacterDType_node=Tree("CharacterDType",CharacterDType_children)
    CharacterDType_dict['node']=CharacterDType_node
    CharacterDType_dict['index']=dict2['index']
    return CharacterDType_dict

def CharacterDType2(i):
    CharacterDType2_dict=dict()
    CharacterDType2_children=[]
    last_index=i
    if i < len(Tokens):
        temp=Tokens[i].to_dict()
        if temp['token_type']==Token_type.openParenthesis :
            match1=Match(Token_type.openParenthesis,i)
            CharacterDType2_children.append(match1['node'])
            match2=Match(Token_type.Len,match1['index'])
            CharacterDType2_children.append(match2['node'])
            match3=Match(Token_type.EqualOp,match2['index'])
            CharacterDType2_children.append(match3['node'])
            dict4=IdorConst(match3['index'])
            CharacterDType2_children.append(dict4['node'])
            match5=Match(Token_type.closeParenthesis,dict4['index'])
            CharacterDType2_children.append(match5['node'])
            last_index=match5['index']
        else :
            CharacterDType2_dict['node']=None
            CharacterDType2_dict['index']=i
            return CharacterDType2_dict
    else :
        match1=Match(Token_type.Error,i)
        CharacterDType2_children.append(match1['node'])
        last_index=match1['index']
    CharacterDType2_node=Tree("CharacterDType2",CharacterDType2_children)
    CharacterDType2_dict['node']=CharacterDType2_node
    CharacterDType2_dict['index']=last_index
    return CharacterDType2_dict

def Statements(i):
    Statements_dict=dict()
    Statements_children=[]
    dict1=Statement(i)
    Statements_children.append(dict1['node'])
    dict2=Statements2(dict1['index'])
    Statements_children.append(dict2['node'])
    Statements_node=Tree("Statements",Statements_children)
    Statements_dict['node']=Statements_node
    Statements_dict['index']=dict2['index']
    return Statements_dict

def Statements2(i):
    Statements2_dict=dict()
    Statements2_children=[]
    last_index=i
    if i < len(Tokens):
        temp=Tokens[i].to_dict()
        #################################################
        if temp['token_type'] in [Token_type.identifier,Token_type.Print,Token_type.read,Token_type.If,Token_type.do] :
            dict1=Statement(i)
            Statements2_children.append(dict1['node'])
            dict2=Statements2(dict1['index'])
            Statements2_children.append(dict2['node'])
            last_index=dict2['index']
            
        else :
            Statements2_dict['node']=None
            Statements2_dict['index']=i
            return Statements2_dict
    else :
        match1=Match(Token_type.Error,i)
        Statements2_children.append(match1['node'])
        last_index=match1['index']
    Statements2_node=Tree("Statements2",Statements2_children)
    Statements2_dict['node']=Statements2_node
    Statements2_dict['index']=last_index
    return Statements2_dict

def Statement(i) :
    Statement_dict=dict()
    Statement_children=[]
    last_index=i
    if i < len(Tokens):
        temp=Tokens[i].to_dict()
        if temp['token_type']==Token_type.identifier :
            dict1=Assignment(i)
            Statement_children.append(dict1['node'])
            last_index=dict1['index']
        elif temp['token_type']==Token_type.Print :
            dict1=Print(i)
            Statement_children.append(dict1['node'])
            last_index=dict1['index']
        elif temp['token_type']==Token_type.read :
            dict1=Read(i)
            Statement_children.append(dict1['node'])
            last_index=dict1['index']
        elif temp['token_type']
        else :
            Statement_dict['node']=None
            Statement_dict['index']=i
            return Statement_dict
    else :
        match1=Match(Token_type.Error,i)
        Statement_children.append(match1['node'])
        last_index=match1['index']



# GUI

In [ ]:
#GUI
root= tk.Tk()
canvas1 = tk.Canvas(root, width=400, height=300, relief='raised')
canvas1.pack()
label1 = tk.Label(root, text='Scanner Phase')
label1.config(font=('helvetica', 14))
canvas1.create_window(200, 25, window=label1)
label2 = tk.Label(root, text='Source code:')
label2.config(font=('helvetica', 10))
canvas1.create_window(200, 100, window=label2)

entry1 = tk.Entry(root) 
canvas1.create_window(200, 140, window=entry1)

def Scan():
    x1 = entry1.get()
    find_token(x1)
    df=pandas.DataFrame.from_records([t.to_dict() for t in Tokens])
    #print(df)
      
    #to display token stream as table
    dTDa1 = tk.Toplevel()
    dTDa1.title('Token Stream')
    dTDaPT = pt.Table(dTDa1, dataframe=df, showtoolbar=True, showstatusbar=True)
    dTDaPT.show()
    # start Parsing
    Node=Parse()
     
    
    # to display errorlist
    df1=pandas.DataFrame(errors)
    dTDa2 = tk.Toplevel()
    dTDa2.title('Error List')
    dTDaPT2 = pt.Table(dTDa2, dataframe=df1, showtoolbar=True, showstatusbar=True)
    dTDaPT2.show()
    Node.draw()
    #clear your list
    
    #label3 = tk.Label(root, text='Lexem ' + x1 + ' is:', font=('helvetica', 10))
    #canvas1.create_window(200, 210, window=label3)
    
    #label4 = tk.Label(root, text="Token_type"+x1, font=('helvetica', 10, 'bold'))
    #canvas1.create_window(200, 230, window=label4)
    
    
button1 = tk.Button(text='Scan', command=Scan, bg='brown', fg='white', font=('helvetica', 9, 'bold'))
canvas1.create_window(200, 180, window=button1)
root.mainloop()